In [76]:
from keras.layers import Input, Concatenate, Add, Reshape, Flatten, Lambda
from keras.models import Model
from models import Conv1D, Dense
import keras.backend as K

In [91]:
hand = Input((13,4))  # zero everywhere and 1 for your cards
board = Input((3,13,4))  # 3 rounds of board: flop [0,:,:], turn [1,:,:] and river [2,:,:]
pot = Input((1,))
stack = Input((1,))  # @todo: include this in the neural network
opponent_stack = Input((1,))  # @todo: include this in the neural network
blinds = Input((2,))  # small, big blinds
dealer = Input((1,))  # 1 if you are the dealer, 0 otherwise
opponent_model = Input((2,))  # tendency to raise, number of hands played: 2 numbers between 0 and 1
preflop_plays = Input((6, 5, 2))  # 6 plays max (check then 5 times raises), 5 possible actions (check,bet,call,raise,all-in), 2 players
flop_plays = Input((6, 5, 2)) 
turn_plays = Input((6, 5, 2)) 
river_plays = Input((6, 5, 2))
# value_of_hand = Input((2,))  # combination_type, rank_in_this_type

In [92]:
hidden_dim = 10
n_actions = 20

Processing board and hand specifically to detect flush and straight

In [93]:
# note: these may need to be wrapped by Lambda layer
color_hand = Lambda(lambda x: K.sum(x, 1))(hand)
color_board = Lambda(lambda x: K.sum(x, (1,2)))(board)
kinds_hand = Lambda(lambda x: K.sum(x, 2))(hand)
kinds_board = Lambda(lambda x: K.sum(x, (1,3)))(board)

colors = Concatenate(2)([Reshape((4,1))(color_hand), Reshape((4,1))(color_board)])
kinds = Concatenate(2)([Reshape((13,1))(kinds_hand), Reshape((13,1))(kinds_board)])

In [94]:
kinds = Conv1D(5,1, activation="prelu")(kinds)
kinds = Conv1D(1, 1, activation='prelu')(
    Concatenate(-1)([
        Conv1D(1, 3)(kinds),
        Conv1D(3, 3, dilation_rate=2)(kinds)
    ])
)
kinds = Dense(hidden_dim, activation='prelu', BN=True)(Flatten()(kinds))

In [95]:
colors = Conv1D(1, 1, activation='prelu')(colors)
colors = Dense(hidden_dim, activation='prelu', BN=True)(Flatten()(colors))

Process board only

In [96]:
board_ = Dense(hidden_dim, activation='prelu', BN=True)(Flatten()(board))
board_ = Dense(hidden_dim, activation='prelu', BN=True)(board_)

Process board and hand together

In [97]:
bh = Dense(hidden_dim, activation='prelu', BN=True)(Concatenate()([Flatten()(board), Flatten()(hand)]))
bh = Dense(hidden_dim, activation='prelu', BN=True)(bh)

In [98]:
cards = Dense(hidden_dim, activation='prelu', BN=True)(Concatenate()([kinds, colors, board_, bh]))
cards = Dense(hidden_dim, activation='prelu', BN=True)(cards)

Add pot, position, blinds

In [99]:
situation_without_opponent = Add()([cards, 
                                    Dense(hidden_dim, activation='prelu', BN=True)(pot),
                                    Dense(hidden_dim, activation='prelu', BN=True)(blinds),
                                    Dense(hidden_dim, activation='prelu', BN=True)(dealer)
                                   ])
situation_without_opponent = Dense(hidden_dim, activation='prelu', BN=True)(situation_without_opponent)
situation_without_opponent = Dense(hidden_dim, activation='prelu', BN=True)(situation_without_opponent)

# maybe this should only be a function of the cards
value_of_hand = Dense(hidden_dim, activation='sigmoid', BN=True)(situation_without_opponent)

Process plays

In [100]:
processed_preflop = Dense(hidden_dim, activation='prelu', BN=True)(Flatten()(preflop_plays))
processed_flop = Dense(hidden_dim, activation='prelu', BN=True)(Flatten()(flop_plays))
processed_turn = Dense(hidden_dim, activation='prelu', BN=True)(Flatten()(turn_plays))
processed_river = Dense(hidden_dim, activation='prelu', BN=True)(Flatten()(river_plays))
processed_opponent = Dense(hidden_dim, activation='prelu', BN=True)(opponent_model)
plays = Dense(hidden_dim, activation='prelu', BN=True)(Add()([processed_preflop, 
                                                     processed_flop, 
                                                     processed_turn, 
                                                     processed_river,
                                                     processed_opponent
                                                    ]))
plays = Dense(hidden_dim, activation='prelu', BN=True)(plays)

In [101]:
situation_with_opponent = Dense(hidden_dim, activation='prelu', BN=True)(Concatenate()([plays, situation_without_opponent]))
situation_with_opponent = Dense(hidden_dim, activation='prelu', BN=True)(situation_with_opponent)

In [102]:
Q_values = Dense(n_actions, activation=None, BN=True)(situation_with_opponent)

In [103]:
Q = Model([hand, board, pot, blinds, dealer, opponent_model, preflop_plays, flop_plays, turn_plays, river_plays], [value_of_hand, Q_values])

In [104]:
Q.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_23 (InputLayer)            (None, 13, 4)         0                                            
____________________________________________________________________________________________________
input_24 (InputLayer)            (None, 3, 13, 4)      0                                            
____________________________________________________________________________________________________
lambda_7 (Lambda)                (None, 13)            0           input_23[0][0]                   
____________________________________________________________________________________________________
lambda_8 (Lambda)                (None, 13)            0           input_24[0][0]                   
___________________________________________________________________________________________

batch_normalization_97 (BatchNor (None, 10)            40          dense_65[0][0]                   
____________________________________________________________________________________________________
batch_normalization_98 (BatchNor (None, 10)            40          dense_66[0][0]                   
____________________________________________________________________________________________________
batch_normalization_99 (BatchNor (None, 10)            40          dense_67[0][0]                   
____________________________________________________________________________________________________
batch_normalization_103 (BatchNo (None, 10)            40          dense_71[0][0]                   
____________________________________________________________________________________________________
batch_normalization_104 (BatchNo (None, 10)            40          dense_72[0][0]                   
___________________________________________________________________________________________

In [105]:
from keras.optimizers import Adam

Q.compile(Adam(), ['mse', 'mse'], loss_weights=[1., 10.])